In [1]:
import numpy as np
import time

In [2]:
# Import 2QAN compiler passes
from py2qan import BenchArch
from py2qan import HeuristicMapper
from py2qan import QuRouter

In [3]:
# Import qiskit 
import qiskit
from qiskit import transpile, QuantumCircuit

## Run 2QAN compiler passes

In [4]:
def qs_compiler(qasm, coupling_map, qaoa=True, layers=1, trials=1, mapper='qap', bgate='rzz', params=None):
    qs_circ = None
    qs_swap = (0, 0) # the number of swaps in the format (#swaps,#swaps merged with circuit gate)
    qs_g2 = 0 # the number of two-qubit gates without decomposition
    # Perform qubit mapping, routing, and scheduling only, without gate decomposition
    for trial in range(trials):
        # Both QAP and Qiskit mappers output inital qubit maps randomly, 
        # one can run the mapper several times to achieve better compilation results
        # Initial qubit mapping 
        start = time.time()
        hmapper = HeuristicMapper(qasm, coupling_map=coupling_map)
        if mapper == 'qap':
            # The default mapper based on Quadratic Assignment Problem
            init_map, cost = hmapper.run_qap(num_iter=200, lst_len=20)
        elif mapper == 'qiskit':
            # The mapper in Qiskit
            init_map = hmapper.run_qiskit(max_iterations=5)
        end = time.time()
        print("Mapper run time: ", end - start)
        # init_map = {circuit qubit index:device qubit index}
        print('The initial qubit map is \n', init_map)

        # Routing and scheduling, takes init_map as input
        router = QuRouter(qasm, init_map=init_map, coupling_map=coupling_map)
        if qaoa:
            # For QAOA, different layers have different gate parameters
            qs_circ0, swaps1 = router.run_qaoa(layers=layers, gammas=params[layers-1][:layers], betas=params[layers-1][layers:], msmt=True) 
        else:
            # For quantum simulation circuits, we assume each layer has the same time steps
            qs_circ0, swaps1 = router.run(layers=layers, msmt='True')
        # qs_circ0 is the routed circuit without gate decomposition
        # swaps1 is a tuple=(#swaps,#swaps merged with circuit gate)

        # Two-qubit gate count and swap count
        qs_circ1 = transpile(qs_circ0, basis_gates=None, optimization_level=3)
        g2_count1 = 0
        if bgate in qs_circ1.count_ops():
            g2_count1 += qs_circ1.count_ops()[bgate]
        if 'unitary' in qs_circ1.count_ops():
            g2_count1 += qs_circ1.count_ops()['unitary']
        if 'swap' in qs_circ1.count_ops():
            g2_count1 += qs_circ1.count_ops()['swap']
        if trial == 0:
            qs_circ = qs_circ1
            qs_swap = swaps1
            qs_g2 = g2_count1 
        elif g2_count1 < qs_g2:
            qs_circ = qs_circ1
            qs_swap = swaps1
            qs_g2 = g2_count1
        print(g2_count1, qs_swap)
    return qs_circ, qs_swap, qs_g2

def qiskit_decompose(circ, basis_gates=['id', 'rz', 'u3', 'u2', 'cx', 'reset'], bgate='cx'):
    # Perform gate decomposition and optimization into cx gate set
    # For decomposition into other gate sets, e.g., the SYC, sqrt iSWAP, iSWAP, 
    # one can use Google Cirq for decomposition or the NuOp (https://github.com/prakashmurali/NuOp) decomposer
    decom_g2 = 0
    decom_circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)
    if bgate in decom_circ.count_ops():
        decom_g2 += decom_circ.count_ops()[bgate] 
    if 'unitary' in decom_circ.count_ops().keys():
        decom_g2 += decom_circ.count_ops()['unitary']
    return decom_circ, decom_g2

## Tests

In [5]:
import os
import pickle as pkl
# Benchmarks
qaoa = True
# QAOA benchmarks
# OpenQASM circuits here only contain one layer/depth
with open(os.path.join('qaoa_qasms.pkl'), 'rb') as f:
    qasms = pkl.load(f)
# The parameters here include gammas for rzz and betas for rx in 4 layers
with open(os.path.join('qaoa_params.pkl'), 'rb') as f:
    params = pkl.load(f)
    
param = None
idx = -2  # circuit id
c_qasm = qasms[idx]
if qaoa:
    param = params[idx]
test_circ = qiskit.QuantumCircuit.from_qasm_str(c_qasm)

In [6]:
# Device information
# gate set, assume cx as the native two-qubit gate
basis_gates = ['id', 'rz', 'u3', 'u2', 'cx', 'reset']

# topology, assume grid architecture as an example
qn = len(test_circ.qubits)
dx = int(np.sqrt(qn))
print('The number of qubits is ', qn)
if dx*dx >= qn:
    lattice_xy = (dx, dx)
elif dx*(dx+1) >= qn:
    lattice_xy = (dx, dx+1)
elif dx*(dx+2) >= qn:
    lattice_xy = (dx, dx+2)
grid_topology = BenchArch(c_qasm, lattice_xy=lattice_xy).topology
coupling_map = [list(edge) for edge in list(grid_topology.edges)]
coupling_map += [[edge[1], edge[0]] for edge in list(grid_topology.edges)]

The number of qubits is  20


## QAP mapper + 2QAN routing&scheduling

In [ ]:
qs_circ, qs_swap, qs_g2 = qs_compiler(c_qasm, coupling_map, qaoa=qaoa, layers=1, trials=5, bgate='rzz', params=param)
print('The number of SWAPs: ', qs_swap, qs_g2)

Mapper run time:  13.592903852462769
The initial qubit map is 
 {0: 4, 1: 7, 2: 18, 3: 11, 4: 16, 5: 15, 6: 12, 7: 9, 8: 2, 9: 1, 10: 17, 11: 3, 12: 5, 13: 0, 14: 14, 15: 10, 16: 13, 17: 6, 18: 8, 19: 19}
36 (6, 0)
Mapper run time:  13.985751628875732
The initial qubit map is 
 {0: 10, 1: 14, 2: 13, 3: 19, 4: 4, 5: 18, 6: 0, 7: 1, 8: 7, 9: 2, 10: 12, 11: 3, 12: 5, 13: 6, 14: 17, 15: 15, 16: 8, 17: 11, 18: 9, 19: 16}
33 (7, 4)
Mapper run time:  14.517908811569214
The initial qubit map is 
 {0: 12, 1: 10, 2: 2, 3: 11, 4: 1, 5: 7, 6: 5, 7: 9, 8: 18, 9: 13, 10: 0, 11: 14, 12: 17, 13: 16, 14: 3, 15: 15, 16: 4, 17: 19, 18: 8, 19: 6}
33 (7, 4)


In [ ]:
qs_circ2, qs_g2 = qiskit_decompose(qs_circ, bgate='cx', basis_gates=basis_gates)
print('The number of CNOTs: ', qs_g2)

## Qiskit SABRE mapper + 2QAN routing&scheduling

In [ ]:
ibm_circ, ibm_swap, ibm_g2 = qs_compiler(c_qasm, coupling_map, qaoa=qaoa, layers=1, trials=5, mapper='qiskit', bgate='rzz', params=param)
print('The number of SWAPs: ', ibm_swap, ibm_g2)

In [ ]:
ibm_circ2, ibm_g2 = qiskit_decompose(ibm_circ, bgate='cx', basis_gates=basis_gates)
print('The number of CNOTs: ', ibm_g2)

## Qiskit compiler

In [ ]:
# test_circ only has one layer in the given example
qiskit_circ = transpile(test_circ, basis_gates=basis_gates, coupling_map=coupling_map, optimization_level=3)
print('The number of CNOTs: ', qiskit_circ.count_ops()['cx'])